In [1]:
!which pip

/Users/snirlugassy/School/data-science-lab-project/venv/bin/pip


In [2]:
import re
import sys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk

data_path = '/Users/snirlugassy/Downloads/data_chunk_0.csv'

In [3]:
data = pd.read_csv(data_path, nrows=10000, index_col='id')
data

,url,text,website,country,region,locality,founded,industry,size,company_name,linkedin
id,,,,,,,,,,,
0,https://www.benefitsboard.com/,Benefits Board Benefits Board HOMEJOIN THE PL...,benefitsboard.com,united states,tennessee,cleveland,NaN,marketing and advertising,1-10,church-of-god-benefits-board,linkedin.com/company/church-of-god-benefits-board
1,https://www.columbialandcare.com/,Home - Columbia Landcare About Environmental S...,columbialandcare.com,united states,missouri,columbia,NaN,environmental services,51-200,columbia-landcare,linkedin.com/company/columbia-landcare
2,https://inregion.com/,In Region Inc. HomeAboutServicesClientsMentori...,inregion.com,united states,new york,new york,2002.0,management consulting,1-10,in-region-inc-,linkedin.com/company/in-region-inc-
3,https://carlsonconsulting.net/,Home About Our Team Careers Services Civil Des...,carlsonconsulting.net,united states,tennessee,memphis,1995.0,civil engineering,11-50,carlson-consulting-engineers,linkedin.com/company/carlson-consulting-engineers
4,https://friedmancpa.com/,Home - Friedman and Associates PA Friedman and...,friedmancpa.com,united states,maryland,owings mills,NaN,accounting,1-10,friedman-and-associates-pa-cpas,linkedin.com/company/friedman-and-associates-p...
...,...,...,...,...,...,...,...,...,...,...,...
9995,https://lowbraudc.com/,ABVセクシープロレス動画を見てください | セクシーグラビアアイドルや有名AV女優たちも続...,lowbraudc.com,united states,district of columbia,washington,2010.0,food & beverages,1-10,low-brau-llc,linkedin.com/company/low-brau-llc
9996,https://youralohawedding.com/,Maui Wedding Planner - Your Aloha Wedding Comp...,youralohawedding.com,united states,hawaii,kihei,2006.0,events services,1-10,your-aloha-wedding-company-inc.,linkedin.com/company/your-aloha-wedding-compan...
9997,https://www.hu3d.co.uk/,HU3D - UK based Exhibition Stand Design & Buil...,hu3d.co.uk,united kingdom,north yorkshire,yorkshire,2015.0,design,1-10,hu3d,linkedin.com/company/hu3d


In [4]:
from nltk.stem.snowball import SnowballStemmer

STOPWORDS = nltk.corpus.stopwords.words('english')
HTML_TAG = re.compile('<.*?>')

class DataProcessing:
    def __init__(self, input_files: list[str], output_file: str, nrows=20000):
        self.files = input_files
    
    def run(self):
        pass


class TextNormalizer:
    def __init__(self):
        self.__stemmer = SnowballStemmer('english', ignore_stopwords=False)

    def _stem(self, t):
        return self.__stemmer.stem(t)

    def normalize(self, text):
        text = re.sub(HTML_TAG, ' ', text)
        text = re.sub('\W', ' ', text)
        text = re.sub('\d+', ' ', text)
        text = re.sub('\s+', ' ', text)
        text = text.lower().strip()
        text = str.join(' ', [self._stem(x) for x in text.split(' ') if len(x) > 1 and x not in STOPWORDS])   
        return text

In [5]:
normalizer = TextNormalizer()

In [6]:
_data = data.sample(5000, random_state=42)
_data['clean'] = _data.text.apply(lambda t: normalizer.normalize(t))
_data['en_prob'] = _data.clean.apply(lambda t: len(re.findall('[A-Za-z\s]', t)) / len(t))
_data[['text', 'clean', 'en_prob']]

,text,clean,en_prob
id,,,
6252,Menu Projects About Publications Contact Desig...,menu project public contact design work lifest...,1.000000
4684,Home - Boise City/Ada County Housing Authoriti...,home bois citi ada counti hous author home pro...,1.000000
1731,HOME | fiskerintl FI about.Daniel Fisker is ...,home fiskerintl fi daniel fisker founder princ...,1.000000
4742,Institute of Terrorism Research & Response Use...,institut terror research respons usernam passw...,1.000000
4521,Masjid Hamza – Islamic Center of South Shore i...,masjid hamza islam center south shore valley s...,0.994898
...,...,...,...
5743,New Patient Special | Chiropractor Aiken SC Sk...,new patient special chiropractor aiken sc skip...,1.000000
6226,Black Wharton Consulting Home About Impact Joi...,black wharton consult home impact join back vi...,1.000000
1973,L&M Taylor Soccer For the Fans Footwear Equipm...,taylor soccer fan footwear equip teamwear goal...,1.000000


In [7]:
_data.query('en_prob<0.5')[['clean', 'en_prob']].shape[0]

18

In [8]:
_data.drop(columns=['text', 'url', 'website', 'linkedin'], inplace=True)
_data.to_csv('test_123.csv')